In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import random
import re
import sklearn.model_selection as sk_ModelSelection
#increase field limit to read embedding
import sys
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!git clone https://github.com/guol1nag/temp.git
torch.cuda.get_device_name()

Cloning into 'temp'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), done.


'Tesla T4'

In [2]:
train_path = r'./temp/toy_X.csv'
test_path = r'./temp/toy_Y.csv'

X = [re.sub("'", "", x.strip('[').strip('[')).split(', ') for x in pd.read_csv(train_path, squeeze = True).tolist()]
y = [re.sub("'", "", x.strip('[').strip('[')).split(', ') for x in pd.read_csv(test_path, squeeze = True).tolist()]

X_train,X_test,y_train,y_test = sk_ModelSelection.train_test_split(X,y,test_size=0.2)

print('number of sentence:',len(X_train))
print('number of tokens e.g.:',len(X_train[0]))


number of sentence: 80
number of tokens e.g.: 988


In [0]:
class LangDict:
  def __init__(self):
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: '<sos>', 1: '<eos>'}
    self.n_words = 2

  def add_article(self, article):
    for word in article:
      self.add_word(word)

  def add_word(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

text_dictionary = LangDict()

for article in X_train:
  text_dictionary.add_article(article)

for article in y_train:
  text_dictionary.add_article(article)

for article in X_test:
  text_dictionary.add_article(article)

for article in y_test:
  text_dictionary.add_article(article)

# pre-train embedding & pre-processing

pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)


In [4]:
%%time

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

# input your pre-train txt path
path = '/content/glove.6B.100d.txt'
embed_dict = {}
with open(path,'r') as f:
  lines = f.readlines()
  for l in lines:
    w = l.split()[0]
    v = np.array(l.split()[1:]).astype('float')
    embed_dict[w] = v

embed_dict['@@_unknown_@@'] = np.zeros(100) # if we use 100 dimension embeddings

!rm -rf glove.6B.zip
!rm -rf glove.6B.50d.txt
!rm -rf glove.6B.100d.txt
!rm -rf glove.6B.200d.txt
!rm -rf glove.6B.300d.txt

len(embed_dict.keys())

--2020-02-29 19:21:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-02-29 19:21:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-02-29 19:21:06--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
%%time
pre_train_weight = []
for word_index in text_dictionary.index2word.keys():
  if word_index != 0:
    word = text_dictionary.index2word[word_index]
    try:
      word_vector = embed_dict[word].reshape(1,-1)
    except:
      word_vector = embed_dict['@@_unknown_@@'].reshape(1,-1) # handle unknown word
    pre_train_weight = np.vstack([pre_train_weight,word_vector])
  else:
    word = text_dictionary.index2word[word_index]
    try:
      word_vector = embed_dict[word].reshape(1,-1)
    except:
      word_vector = embed_dict['@@_unknown_@@'].reshape(1,-1) # handle unknown word
    pre_train_weight = word_vector

#embed = nn.Embedding.from_pretrained(torch.from_numpy(pre_train_weight),freeze=True)
print(pre_train_weight.shape)

(12914, 100)
CPU times: user 9.58 s, sys: 608 ms, total: 10.2 s
Wall time: 10.2 s



# seq2seq

see: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#training-the-model

RNNs are inherently sequential. They are auto-regressive, meaning the input for timestep t contains the output for timestep (t-1), meaning you have to first calculae the output for timestap (t-1).

##### encoder

In [0]:
class Encoder(nn.Module):
  def __init__(self, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()
    self.rnn = nn.GRU(input_size = emb_dim, hidden_size = enc_hid_dim, bidirectional = True)
    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    self.dropout = nn.Dropout(dropout)
    
  def forward(self, enc_input):
    
    #enc_input = [enc_input_len, batch size,emb_dim]
    
    embedded = self.dropout(enc_input)
    
    #embedded = [enc_input_len, batch size, emb_dim]
    
    outputs, hidden = self.rnn(embedded)
            
    #outputs = [enc_input len, batch size, hid dim * num directions]
    #hidden = [n layers * num directions, batch size, hid dim]
    
    #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
    #outputs are always from the last layer
    
    #hidden [-2, :, : ] is the last of the forwards RNN 
    #hidden [-1, :, : ] is the last of the backwards RNN
    
    #initial decoder hidden is final hidden state of the forwards and backwards 
    #  encoder RNNs fed through a linear layer
    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
    
    #outputs = [src len, batch size, enc hid dim * 2]
    #hidden = [batch size, dec hid dim]
    return outputs, hidden

##### decoder

In [0]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()
    
    self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
    self.v = nn.Linear(dec_hid_dim, 1, bias = False)
      
  def forward(self, hidden, encoder_outputs):
    
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]
    
    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]
    
    #repeat decoder hidden state src_len times
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
    
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    
    #hidden = [batch size, src len, dec hid dim]
    #encoder_outputs = [batch size, src len, enc hid dim * 2]
    
    energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
    
    #energy = [batch size, src len, dec hid dim]

    attention = self.v(energy).squeeze(2)
    
    #attention= [batch size, src len]
  
    return F.softmax(attention, dim=1)

In [0]:
class Decoder(nn.Module):
  def __init__(self, output_dim,enc_hid_dim,  dec_hid_dim, dropout, attention):
    super().__init__()

    self.output_dim = output_dim
    self.attention = attention
    
    #self.embedding = nn.Embedding(output_dim, output_dim)
    
    self.rnn = nn.GRU((enc_hid_dim * 2) + output_dim, dec_hid_dim)
    
    self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + output_dim, output_dim)
    
    self.dropout = nn.Dropout(dropout)
      
  def forward(self, dec_input, hidden, encoder_outputs):
          
    #dec_input = [1,batch size,dec_emb dim]
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]
            
    embedded = self.dropout(dec_input)
    
    #embedded = [1, batch size, dec_emb dim]
    
    a = self.attention(hidden, encoder_outputs)

    #a = [batch size, src len]
    
    a = a.unsqueeze(1)
    
    #a = [batch size, 1, src len]
    
    encoder_outputs = encoder_outputs.permute(1, 0, 2)
    
    #encoder_outputs = [batch size, src len, enc hid dim * 2]
    
    weighted = torch.bmm(a, encoder_outputs)
    
    #weighted = [batch size, 1, enc hid dim * 2]
    
    weighted = weighted.permute(1, 0, 2)
    #weighted = [1, batch size, enc hid dim * 2]

    # print('embedded',embedded.size())
    rnn_input = torch.cat((embedded, weighted), dim = 2)
    
    #rnn_input = [1, batch size, (enc hid dim * 2) + dec_emb dim]
        
    output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
    
    #output = [seq len, batch size, dec hid dim * n directions]
    #hidden = [n layers * n directions, batch size, dec hid dim]
    
    #seq len, n layers and n directions will always be 1 in this decoder, therefore:
    #output = [1, batch size, dec hid dim]
    #hidden = [1, batch size, dec hid dim]
    #this also means that output == hidden
    assert (output == hidden).all()
    
    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)
    
    prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
    
    #prediction = [batch size, output dim]
    
    return prediction, hidden.squeeze(0)

seq 2 seq

In [0]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
      super().__init__()
      
      self.encoder = encoder
      self.decoder = decoder
      self.device = device
      
  def forward(self, seq2seq_input: '[seq_len, batch size,Enc_emb_dim]',\
               target: '[trg_len, batch size,output_dim]',\
               teacher_forcing_ratio = 0.5):
      
      #seq2seq_input = [seq_len, batch size,Enc_emb_dim]
      #target = [trg_len, batch size,output_dim]
      #teacher_forcing_ratio is probability to use teacher forcing
      #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time

      batch_size = seq2seq_input.shape[1]
      trg_len = target.shape[0]
      trg_vocab_size = self.decoder.output_dim
      
      #tensor to store decoder dec_output
      dec_output = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
      
      #encoder_outputs is all hidden states of the input sequence, back and forwards
      #hidden is the final forward and backward hidden states, passed through a linear layer
      encoder_outputs, hidden = self.encoder(seq2seq_input)
              
      # check: make dimension consistent 
      dec_input = target[0]
      dec_input = dec_input.unsqueeze(0)
      
      # print('dec_input dim:',dec_input.size())
      
      for t in range(1, trg_len):
        #insert dec_input token embedding, previous hidden state and all encoder hidden states
        #receive output tensor (predictions) and new hidden state
        output, hidden = self.decoder(dec_input, hidden, encoder_outputs)
        
        #place predictions in a tensor holding predictions for each token
        dec_output[t] = output
        
        #decide if we are going to use teacher forcing or not
        teacher_force = random.random() < teacher_forcing_ratio
        
        #get the highest predicted token from our predictions
        top1 = output

        #if teacher forcing, use actual next token as next input
        #if not, use predicted token
        dec_input = target[t] if teacher_force else top1
        dec_input = dec_input.unsqueeze(1).float()

      return dec_output

In [0]:
class utility(): #utility functions
  @staticmethod
  def article_list_2_index_tensor(article_list: list) -> torch.Tensor:
    article_list = sorted(article_list,key=lambda x:len(x),reverse=True)
    longest = len(article_list[0])
    for i,article in enumerate(article_list):
      index = torch.Tensor([text_dictionary.word2index[token] for token in article if token is str])
      padding = longest - len(index)
      index = torch.cat([index,torch.ones(padding)]).long().view(1,-1)   ########## TODO: need to change padding moode
      index_list = (index if i == 0 else torch.cat((index_list,index),dim=0))
    return index_list
    
  @staticmethod
  def batcher(iterable: torch.Tensor,\
              label: torch.Tensor, \
              batch_size: int = 1) -> torch.Tensor:
    l = len(iterable)
    for batch in range(0, l, batch_size):
        yield (iterable[batch:min(batch + batch_size, l)],label[batch:min(batch + batch_size, l)])

  @staticmethod
  def Loss_opt(model, learning_rate):
    Lossfunc = torch.nn.CosineEmbeddingLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return(Lossfunc, optimizer)

  @staticmethod
  def count_parameters(model):
      return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [49]:
grid = {'max_epochs':3,
        'batch_size':1,
        'leanring_rate':1e-3,
        'clip':10
      }

##### model ######
OUTPUT_DIM = 100
ENC_EMB_DIM = 100
#DEC_EMB_DIM = 1
ENC_HID_DIM = 128
DEC_HID_DIM = 128
ENC_DROPOUT = 0
DEC_DROPOUT = 0

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(output_dim = OUTPUT_DIM,  enc_hid_dim = ENC_HID_DIM, dec_hid_dim = DEC_HID_DIM, dropout = DEC_DROPOUT,attention= attn)
model = Seq2Seq(enc, dec, device).to(device)

###### embedding layer 
embedding_layer = nn.Embedding.from_pretrained(torch.from_numpy(pre_train_weight),freeze=True).to(device)

###### optimizer, lossfunction 
lossfunction,optimiser = utility.Loss_opt(model,grid['leanring_rate'])


print(f'The model has {utility.count_parameters(model):,} trainable parameters')
print('')
model

The model has 494,068 trainable parameters



Seq2Seq(
  (encoder): Encoder(
    (rnn): GRU(100, 128, bidirectional=True)
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=384, out_features=128, bias=True)
      (v): Linear(in_features=128, out_features=1, bias=False)
    )
    (rnn): GRU(356, 128)
    (fc_out): Linear(in_features=484, out_features=100, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
)

In [0]:
def training(grid,X_train,y_train,model,lossfunction,optimiser):

  #collect statistics
  train_loss =[]
  val_loss =[]
  best_loss = float('inf')

  for epoch in range(grid['max_epochs']):
    trainloss = 0

    for local_batch, local_labels in utility.batcher(X_train,y_train,grid['batch_size']):
      local_batch = utility.article_list_2_index_tensor(local_batch).to(device)
      local_labels = utility.article_list_2_index_tensor(local_labels).to(device)

      # pass through embedding layer to become vectors
      local_batch = embedding_layer(local_batch).float().transpose(0,1)
      local_labels = embedding_layer(local_labels).float().transpose(0,1)

      print('input tensor are:')
      print(local_batch.size())
      print(local_labels.size())
      
      optimiser.zero_grad()
      local_output = model(seq2seq_input = local_batch,target = local_labels,\
                teacher_forcing_ratio=1)
      
      local_output = local_output.squeeze(1)
      local_labels = local_labels.squeeze(1)

      print('output are:')
      print(local_output.size())
      print(local_labels.size())

      loss = lossfunction(local_output,local_labels,torch.Tensor([1]).to(device)) # TODO figure out how to use this in matrix

      print('loss:')
      print(loss.item())
      print('')
      
      trainloss += loss.item()

      loss.backward()
      optimiser.step()

      for i,param in enumerate(model.parameters()):
        if i == 12:
          print('')
          print('accu grad')
          print('')
          print(param.grad)
          print('')
          print('weight of a layer')
          print('')
          print(model.decoder.fc_out.weight)
          break

    print(trainloss)
    train_loss.append(trainloss)
  return(train_loss)

In [51]:
training(grid,X_train,y_train,model,lossfunction,optimiser)

input tensor are:
torch.Size([988, 1, 100])
torch.Size([7, 1, 100])
output are:
torch.Size([700])
torch.Size([700])
loss:
0.0015522119356319308


accu grad

tensor([[-3.5280e-10, -7.3237e-09,  6.0441e-10,  3.2695e-09, -1.1592e-08,
          1.1562e-08,  2.5941e-09,  1.1581e-08, -2.4290e-09,  1.1274e-08,
          6.7235e-09,  4.5446e-09, -9.9300e-09, -4.8586e-10,  1.5775e-09,
         -7.3519e-09,  4.6500e-09,  1.9762e-08, -5.2684e-09,  2.5188e-09,
          1.5241e-09,  1.2968e-08,  5.4733e-09,  8.5555e-09,  2.6883e-09,
          2.1295e-09,  5.9762e-09, -4.6316e-10, -5.5479e-09, -3.4194e-10,
         -8.6947e-09, -6.0616e-09, -4.5247e-09, -2.9444e-09, -3.9826e-09,
         -1.9204e-09, -2.9745e-10, -5.7588e-09, -2.3542e-09, -1.4005e-09,
          9.2954e-10, -9.6939e-09, -4.4679e-09,  1.4347e-08,  2.6195e-09,
         -1.3070e-09, -1.6132e-10,  9.5288e-09,  2.1003e-09,  6.9331e-09,
          1.3579e-08, -5.3485e-09,  1.1747e-08, -2.9821e-09,  2.9438e-09,
         -1.1250e-08, -1.1611

KeyboardInterrupt: ignored

In [0]:
def training(model, X_train,y_train, optimiser, lossfunction, clip):
  model.train()
  epoch_loss = 0

  local_batch, local_labels = X_train.unsqueeze(1).to(device), y_train.unsqueeze(1).to(device) 
  local_labels = local_labels.unsqueeze(2)

  # print('input')
  # print(local_batch.size())
  # print(local_labels.size())

  ### forward pass
  optimiser.zero_grad()
  
  local_output = model(seq2seq_input=local_batch, target=local_labels, teacher_forcing_ratio = 1)
  local_output = local_output.view(-1)[1:]
  local_labels = local_labels.view(-1)[1:]

  # print('output:')
  # print(local_output.size())
  # print(local_labels.size())
  loss = lossfunction(local_output, local_labels)

  ### backward pass
  loss.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
  optimiser.step()
  epoch_loss += loss.item()
  return epoch_loss 

def evaluate(model, X_test,y_test, lossfunction):
  
  model.eval()
  epoch_loss = 0
  with torch.no_grad():
    local_batch, local_labels = X_test.unsqueeze(1).to(device), y_test.unsqueeze(1).to(device) 
    local_labels = local_labels.unsqueeze(2)
          
    local_output = model(seq2seq_input=local_batch, target=local_labels, teacher_forcing_ratio =0)

    local_output = local_output.view(-1)[1:]
    local_labels = local_labels.view(-1)[1:]
  
    loss = lossfunction(local_output, local_labels)
    epoch_loss += loss.item()
  return epoch_loss

def running(grid,model,optimiser,lossfunction,X_train,y_train,X_test,y_test):

  best_valid_loss = float('inf')

  for epoch in range(grid['max_epochs']):

    train_loss = training(model, X_train, y_train, optimiser, lossfunction, grid['clip'])
    valid_loss = evaluate(model, X_test,y_test, lossfunction)

    if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      print(f'Epoch: {epoch+1}:')
      print(f'Train Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')

  return best_valid_loss


accu grad

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], device='cuda:0')

weight of a layer

Parameter containing:
tensor([[ 0.0535, -0.0186,  0.0476,  ...,  0.0609,  0.0702,  0.0300],
        [-0.0243, -0.0043,  0.0394,  ..., -0.0671, -0.0652, -0.0526],
        [-0.0673,  0.0480, -0.0358,  ...,  0.0633,  0.0214,  0.0235],
        ...,
        [-0.0438,  0.0349,  0.0179,  ...,  0.0641,  0.0388, -0.0305],
        [ 0.0624,  0.0196,  0.0396,  ..., -0.0652, -0.0617,  0.0620],
        [ 0.0142, -0.0122,  0.0099,  ...,  0.0294, -0.0241, -0.0059]],
       device='cuda:0', requires_grad=True)

accu grad

KeyboardInterrupt: ignored

In [0]:
grid = {'max_epochs':512,
        'batch_size':-1, # full batch
        'learning_rate':1e-3,
        'clip':10
      }

##### model ######
OUTPUT_DIM = 1
ENC_EMB_DIM = 21
#DEC_EMB_DIM = 1
ENC_HID_DIM = 32
DEC_HID_DIM = 32
ENC_DROPOUT = 0
DEC_DROPOUT = 0

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(output_dim = OUTPUT_DIM,  enc_hid_dim = ENC_HID_DIM, dec_hid_dim = DEC_HID_DIM, dropout = DEC_DROPOUT,attention= attn)

model = Seq2Seq(enc, dec, device).to(device)
optimiser = optim.Adam(model.parameters(),lr = grid['learning_rate'])
lossfunction = nn.MSELoss().to(device)
model.apply(utility.init_weights)
model

In [0]:
running(grid,model,optimiser,lossfunction,X_train,y_train,X_test,y_test)